# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on a small subset femnist images, comprising 3 classes, and all images are 28 x 28 pixels. The dataset can be downloaded from [here](https://drive.google.com/drive/folders/1FIfdQZHv0W7ef1ByC0fQssvSrgPMFaqL?usp=sharing). The task to be carried out will be a multi-class prediction.

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, follow the instructions on [pysyft_ttp readme](https://gitlab.int.aisingapore.org/aims/federatedlearning/pysyft_worker/-/tree/dev) and [pysyft_worker readme](https://gitlab.int.aisingapore.org/aims/federatedlearning/pysyft_ttp/-/tree/plan_integrations) on how to build the required docker images for TTP and Workers.

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory femnist/data1>:/worker/data -v <directory femnist/outputs_1>:/worker/outputs --name worker_1 worker:pysyft_demo```

**Worker 2**

```docker run -v <directory femnist/data2>:/worker/data -v <directory femnist/outputs_2>:/worker/outputs --name worker_2 worker:pysyft_demo```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory femnist/mlflow_test>:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo```


Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

## Initializing Synergos driver

In [1]:
from synergos import Driver

host = "localhost"
port = 5000

driver = Driver(host=host, port=port)

## Phase 1: CONNECT - Submitting TTP & Participant metadata #

#### 1A. TTP controller creates a project

Create a project which is a classification task with action="classify", if the project is a regression task thenn action="regression".

In [2]:
# 1A. TTP controller creates a project
driver.projects.create(
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': []
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'project_id': 'test_project'},
  'relations': {'Experiment': [{'doc_id': '1',
     'kind': 'Experiment',
     'key': {'project_id': 'test_project', 'expt_id': 'test_experiment'},
     'model': [{'is_input': True,
       'structure': {'in_channels': 1,
        'kernel_size': 3,
        'out_channels': 4,
        'padding': 1,
        'stride': 1},
       'l_type': 'Conv2d',
       'activation': 'relu'},
      {'is_input': False, 'l_type': 'Flatten'},
      {'is_input': False,
       'structure': {'bias': True, 'in_features': 3136, 'out_features': 1},
       'l_type': 'Linear',
       'activation': 'softmax'}]}],
   'Run': [{'doc_id': '1',
     'kind': 'Run',
     'key': {'project_id': 'test_project',
      'expt_id': 'test_experiment',
      'run_id': 'test_run'},
     'rounds': 2,
     'epochs': 1,
     'lr': 0.001,
     'lr_decay': 0.1,
     'weight_decay': 0.0,
     'seed': 42,
     'precision_fractional': 5,
     'mu': 0.1,
   

#### 1B. TTP controller creates an experiment

In [3]:
# 1B. TTP controller creates an experiment
driver.experiments.create(
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "in_channels": 1, 
                "out_channels": 4, # [N, 4, 28, 28]
                "kernel_size": 3,
                "stride": 1,
                "padding": 1
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        # ------------------------------
        {
            "activation": "softmax",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4 * 28 * 28,
                "out_features": 1
            }
        }

    ]
)

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'project_id': 'test_project'},
 'data': {'created_at': '2020-09-23 11:15:04 N',
  'key': {'expt_id': 'test_experiment', 'project_id': 'test_project'},
  'model': [{'activation': 'relu',
    'is_input': True,
    'l_type': 'Conv2d',
    'structure': {'in_channels': 1,
     'kernel_size': 3,
     'out_channels': 4,
     'padding': 1,
     'stride': 1}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Flatten',
    'structure': {}},
   {'activation': 'softmax',
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 3136, 'out_features': 1}}],
  'relations': {'Run': [{'algorithm': 'FedProx',
     'base_lr': 0.0005,
     'created_at': '2020-09-23 11:06:28 N',
     'criterion': 'NLLLoss',
     'delta': 0.0,
     'epochs': 1,
     'is_snn': False,
     'key': {'expt_id': 'test_experiment',
      'project_id': 'test_project',
      'run_id': 'te

#### 1C. TTP controller creates a run

In [4]:
driver.runs.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'project_id': 'test_project',
   'expt_id': 'test_experiment',
   'run_id': 'test_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'project_id': 'test_project', 'expt_id': 'test_experiment'}}

#### 1D. Participants registers their servers' configurations

In [5]:
driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [{'doc_id': '2',
     'kind': 'Registration',
     'key': {'project_id': 'test_project',
      'participant_id': 'test_participant_2'},
     'role': 'host'}],
   'Tag': [{'doc_id': '2',
     'kind': 'Tag',
     'key': {'project_id': 'test_project',
      'participant_id': 'test_participant_2'},
     'train': [['train']],
     'evaluate': [],
     'predict': []}]},
  'id': 'test_participant_2',
  'host': '172.17.0.3',
  'port': 8020,
  'log_msgs': True,
  'verbose': True,
  'f_port': 5000},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

#### 1E. Participants registers their role in a specific project

In [6]:
driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'project': {'incentives': {'tier_2': [], 'tier_1': []}, 'start_at': None},
  'participant': {'id': 'test_participant_2',
   'host': '172.17.0.3',
   'port': 8020,
   'f_port': 5000,
   'log_msgs': True,
   'verbose': True},
  'role': 'host'},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {}}

#### 1F. Participants registers their tags for a specific project

In [7]:
driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_1",
    train=[
        ["train"]
    ],
    evaluate = [["evaluate"]]
)

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["train"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [],
  'predict': []},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

## Phase 2: TRAIN - Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [8]:
driver.alignments.create(project_id="test_project")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'test_project'},
 'data': [{'created_at': '2020-09-23 11:16:28 N',
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project'},
   'link': {'alignment_id': '39dd610afd8e11eaadfd0242ac110004',
    'registration_id': '08c238f2fd8e11ea8b4b0242ac110004',
    'tag_id': '09170a3afd8e11eab1a10242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 1,
   'kind': 'Alignment'},
  {'created_at': '2020-09-23 11:16:28 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project'},
   'link': {'alignment_id': '39e1b8ccfd8e11eaadfd0242ac110004',
    'registration_id': '08b915d8fd8e11eab2620242ac110004',
    'tag_id': '090f993afd8e11ea94c20242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 2,
   'kind': 'Alignment'}]}

#### 2B. Trigger training across the federated grid

In [9]:
model_resp = driver.models.create(
        project_id="test_project",
        expt_id="test_experiment",
        run_id="test_run")

## Phase 3: EVALUATE - Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [11]:
driver.validations.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Validation',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'evaluate': {'statistics': {'accuracy': [0.6629213483146067],
     'roc_auc_score': [0.5],
     'pr_auc_score': [0.6685393258426966],
     'f_score': [0.0],
     'TPRs': [0.0, 1.0, 0.0],
     'TNRs': [1.0, 0.0, 1.0],
     'PPVs': [0.0, 0.3258426966292135, 0.0],
     'NPVs': [0.6629213483146067, 0.0, 0.6629213483146067],
     'FPRs': [0.0, 1.0, 0.0],
     'FNRs': [1.0, 0.0, 1.0],
     'FDRs': [0.0, 0.6741573033707865, 0.0],
     'TPs': [0, 29, 0],
     'TNs': [59, 0, 59],
     'FPs': [0, 60, 0],
     'FNs': [30, 0, 30]},
    'res_path': '/worker/outputs/test_project/test_experiment/test_run/evaluate/inference_statistics_evaluate.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'validations.post',
 'params': {'project_id': 'test_project',
  'expt_id': 'test_experim

#### 3B. Perform prediction(s) of combination(s)

In [12]:
driver.predictions.create(
    tags={"test_project": [["predict"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.29931972789115646],
     'roc_auc_score': [0.5],
     'pr_auc_score': [0.8503401360544218],
     'f_score': [0.0],
     'TPRs': [0.0, 1.0, 0.0],
     'TNRs': [1.0, 0.0, 1.0],
     'PPVs': [0.0, 0.2108843537414966, 0.0],
     'NPVs': [0.29931972789115646, 0.0, 0.9115646258503401],
     'FPRs': [0.0, 1.0, 0.0],
     'FNRs': [1.0, 0.0, 1.0],
     'FDRs': [0.0, 0.7891156462585034, 0.0],
     'TPs': [0, 62, 0],
     'TNs': [88, 0, 268],
     'FPs': [0, 232, 0],
     'FNs': [206, 0, 26]},
    'res_path': '/worker/outputs/test_project/test_experiment/test_run/predict/inference_statistics_predict.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'predictions.post',
 'params': {'project_id': 'test_project',
  'expt_id': 'test_exper